In [1]:
%cd ..

/home/quasar/GitHub/TradeForecast


/home/quasar/GitHub/TradeForecast/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from tradeforecast import Scrapper

ticker = 'AAPL'
scrapper = Scrapper(ticker)

exported_fnames_dict = scrapper.export_historic_data()
exported_fnames_dict

[*********************100%***********************]  1 of 1 completed


{'AAPL': 'AAPL_1d_max_(None-None).csv'}

In [3]:
from tradeforecast.augmentation import DataEntryPoint, Indicators, FeatureEngg, RNNDataset

data_entry = DataEntryPoint(exported_fnames_dict['AAPL'])

indicators = Indicators(data_entry)
indicators.add_macd_sl().add_rsi().add_atr()

features = FeatureEngg(data_entry)
features.add_quarters().add_weeks()

lf = data_entry.data.drop_nulls()
lf.head().collect()

Datetime,Close,High,Low,Open,Volume,MACD_close_12-26,MACD-SL_close_9,RSI_close_14,ATR_close_14,Quarter_sin,Quarter_cos,Week_sin,Week_cos
"datetime[μs, EST]",f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64
1981-01-29 19:00:00 EST,0.126116,0.127232,0.126116,0.127232,46188800,-0.001314,0.000148,23.832346,0.004126,1.0,0.0,0.558647,0.829406
1981-02-01 19:00:00 EST,0.118862,0.11942,0.118862,0.11942,23766400,-0.002705,-0.000491,18.469055,0.004546,1.0,0.0,0.558647,0.829406
1981-02-02 19:00:00 EST,0.123326,0.123884,0.123326,0.123326,19152000,-0.003426,-0.001133,29.702202,0.004609,1.0,0.0,0.652822,0.757511
1981-02-03 19:00:00 EST,0.12779,0.128348,0.12779,0.12779,27865600,-0.003613,-0.001666,39.344816,0.004665,1.0,0.0,0.652822,0.757511
1981-02-04 19:00:00 EST,0.12779,0.128906,0.12779,0.12779,7929600,-0.003719,-0.0021,39.344816,0.00419,1.0,0.0,0.652822,0.757511


In [4]:
from torch.utils.data import DataLoader

dataset_kwargs = {'lf': lf,
                 'non_temporal': data_entry.non_temporal,
                 'temporal': data_entry.temporal,
                 'target': 'Close',
                 'look_back_len': 60,
                 'forecast_len': 5,
                 'split': 0.2}

train_dataset = RNNDataset(train=True, **dataset_kwargs)
test_dataset = RNNDataset(train=False, **dataset_kwargs)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)

In [5]:
from tradeforecast import LSTM
from torch import nn, optim

lstm_kwargs = {'input_size': len(train_dataset.features),
              'hidden_size': 32,
              'n_LSTM': 2,
              'fc_out_size':[],
              'output_size': train_dataset.forecast_len,
              'dropout': 0}

lstm_model = LSTM(**lstm_kwargs)
print(f"The device is set to ---> '{lstm_model.device}'")

The device is set to ---> 'cuda'


In [6]:
lstm_model.train_model(nn.MSELoss, optim.SGD, 1500, train_loader, 0.0001)

Epoch: [1/1500]; Loss: 7.982667
Epoch: [2/1500]; Loss: 7.961636
Epoch: [3/1500]; Loss: 7.942747
Epoch: [4/1500]; Loss: 7.925165
Epoch: [5/1500]; Loss: 7.908195
Epoch: [6/1500]; Loss: 7.891553
Epoch: [7/1500]; Loss: 7.875108
Epoch: [8/1500]; Loss: 7.858792
Epoch: [9/1500]; Loss: 7.842571
Epoch: [10/1500]; Loss: 7.826425
Epoch: [11/1500]; Loss: 7.810341
Epoch: [12/1500]; Loss: 7.794313
Epoch: [13/1500]; Loss: 7.778336
Epoch: [14/1500]; Loss: 7.762406
Epoch: [15/1500]; Loss: 7.746518
Epoch: [16/1500]; Loss: 7.730665
Epoch: [17/1500]; Loss: 7.714838
Epoch: [18/1500]; Loss: 7.699030
Epoch: [19/1500]; Loss: 7.683234
Epoch: [20/1500]; Loss: 7.667444
Epoch: [21/1500]; Loss: 7.651658
Epoch: [22/1500]; Loss: 7.635874
Epoch: [23/1500]; Loss: 7.620092
Epoch: [24/1500]; Loss: 7.604315
Epoch: [25/1500]; Loss: 7.588545
Epoch: [26/1500]; Loss: 7.572784
Epoch: [27/1500]; Loss: 7.557031
Epoch: [28/1500]; Loss: 7.541284
Epoch: [29/1500]; Loss: 7.525540
Epoch: [30/1500]; Loss: 7.509795
Epoch: [31/1500]; L

In [7]:
y, y_preds = lstm_model.test_model(test_loader)

In [8]:
from sklearn.metrics import mean_absolute_percentage_error
from torch import Tensor

y: Tensor = y; y_preds: Tensor = y_preds
y_arr, y_preds_arr = y.cpu().numpy(), y_preds.cpu().numpy()
y_arr.shape, y_preds_arr.shape

((2081, 5), (2081, 5))

In [9]:
test_dataset.inverse_transform(y_arr), test_dataset.inverse_transform(y_preds_arr)

(array([[ 24.857502,  24.664997,  24.335003,  24.1675  ,  25.7375  ],
        [ 24.664997,  24.335003,  24.1675  ,  25.7375  ,  26.084995],
        [ 24.335003,  24.1675  ,  25.7375  ,  26.084995,  26.052502],
        ...,
        [230.76    , 230.56999 , 231.41    , 233.39998 , 233.67    ],
        [230.56999 , 231.41    , 233.39998 , 233.67    , 230.1     ],
        [231.41    , 233.39998 , 233.67    , 230.1     , 225.91    ]],
       dtype=float32),
 array([[ 79.88637 ,  76.27879 ,  78.08257 ,  76.49491 ,  80.79013 ],
        [ 80.27507 ,  76.328285,  78.03215 ,  76.67745 ,  80.96423 ],
        [ 84.34467 ,  78.099686,  80.17482 ,  79.830475,  84.491585],
        ...,
        [290.9504  , 292.72244 , 286.73322 , 284.18796 , 275.49802 ],
        [283.5692  , 285.1446  , 279.38495 , 276.63626 , 268.63135 ],
        [276.87775 , 278.52917 , 272.80762 , 269.96152 , 262.60272 ]],
       dtype=float32))

In [10]:
mean_absolute_percentage_error(y_arr, y_preds_arr)

np.float64(0.5867664911963218)

In [11]:
model_fname = lstm_model.save_model_state(ticker_interval='AAPL_1d')
model_fname

'AAPL_1d_LSTM_14629.pth'